In [14]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import datetime
from langchain_openai import ChatOpenAI
from localschema import AnswerQuestion, ReviseAnswer
from langchain_core.output_parsers.openai_tools import PydanticToolsParser, JsonOutputToolsParser
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv, find_dotenv
from langchain_core.runnables import Runnable, chain
import os


In [2]:
_ = load_dotenv(find_dotenv())
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "default"

In [3]:
pydantic_parser = PydanticToolsParser(tools=[AnswerQuestion])
parser = JsonOutputToolsParser(return_id=True)

In [4]:
# Actor Agent Prompt 
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are expert AI researcher.
Current time: {time}

1. {first_instruction}
2. Reflect and critique your answer. Be severe to maximize improvement.
3. After the reflection, **list 1-3 search queries separately** for researching improvements. Do not include them inside the reflection.
""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Answer the user's question above using the required format."),
    ]
).partial(
    time=lambda: datetime.datetime.now().isoformat(),
)


In [5]:
first_responder_prompt_template = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250 word answer"
)

In [6]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [7]:
first_responder_chain = first_responder_prompt_template | llm.bind_tools(tools=[AnswerQuestion], tool_choice='AnswerQuestion') | pydantic_parser


In [8]:
#playing with the first responder chain
# response = first_responder_chain.invoke({
#     "messages": [HumanMessage("Create a blog post about the plane crash in India that happened in June 2025")]      # this hallucinates
# })
response = first_responder_chain.invoke({
    "messages": [HumanMessage("Write a blog post about the most famous Formula 1 driver of all time")]      
})
print(response)

[AnswerQuestion(answer="When discussing the most famous Formula 1 driver of all time, many enthusiasts point to Michael Schumacher. Renowned for his extraordinary skill, determination, and record-breaking achievements, Schumacher's impact on the sport is unparalleled.\n\nMichael Schumacher, born on January 3, 1969, in Germany, made his Formula 1 debut in 1991. His career spanned over two decades, and he raced for several teams, most notably Benetton and Ferrari. Schumacher's affinity for speed and tactical driving made him a formidable competitor, and he quickly rose to prominence.\n\nSchumacher holds numerous records in Formula 1, including seven World Drivers' Championships—tied with Lewis Hamilton. He also claimed a staggering 91 race victories, many of which came in consecutive seasons, demonstrating his dominance in the sport. \n\nHis tenure at Ferrari is legendary, where he led the team to unprecedented success, contributing to a golden era in F1. Schumacher's meticulous approach

In [ ]:
# Get and print the graph of the chain
graph = first_responder_chain.get_graph()
graph.print_ascii()

       +-------------+         
       | PromptInput |         
       +-------------+         
              *                
              *                
              *                
    +--------------------+     
    | ChatPromptTemplate |     
    +--------------------+     
              *                
              *                
              *                
        +------------+         
        | ChatOpenAI |         
        +------------+         
              *                
              *                
              *                
   +---------------------+     
   | PydanticToolsParser |     
   +---------------------+     
              *                
              *                
              *                
+---------------------------+  
| PydanticToolsParserOutput |  
+---------------------------+  


In [38]:
# Get and print the prompts
print(type(response))
for item in response:
    print(item)

<class 'list'>
answer="When discussing the most famous Formula 1 driver of all time, many enthusiasts point to Michael Schumacher. Renowned for his extraordinary skill, determination, and record-breaking achievements, Schumacher's impact on the sport is unparalleled.\n\nMichael Schumacher, born on January 3, 1969, in Germany, made his Formula 1 debut in 1991. His career spanned over two decades, and he raced for several teams, most notably Benetton and Ferrari. Schumacher's affinity for speed and tactical driving made him a formidable competitor, and he quickly rose to prominence.\n\nSchumacher holds numerous records in Formula 1, including seven World Drivers' Championships—tied with Lewis Hamilton. He also claimed a staggering 91 race victories, many of which came in consecutive seasons, demonstrating his dominance in the sport. \n\nHis tenure at Ferrari is legendary, where he led the team to unprecedented success, contributing to a golden era in F1. Schumacher's meticulous approach 